In [4]:
import dask.dataframe as dd
import pandas as pd
import time

In [5]:
#Reading from a csv
ddf = dd.read_csv(r'D:\DaskData\WeatherEvents.csv',dtype={"ZipCode": "Int64"})

In [9]:
df = pd.read_csv(r'D:\DaskData\WeatherEvents.csv')

In [10]:
#Getting a sneak peak in the data
ddf.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149


In [11]:
#To get the datatypes of columns
ddf._meta.dtypes

EventId              string[pyarrow]
Type                 string[pyarrow]
Severity             string[pyarrow]
StartTime(UTC)       string[pyarrow]
EndTime(UTC)         string[pyarrow]
Precipitation(in)            float64
TimeZone             string[pyarrow]
AirportCode          string[pyarrow]
LocationLat                  float64
LocationLng                  float64
City                 string[pyarrow]
County               string[pyarrow]
State                string[pyarrow]
ZipCode                        Int64
dtype: object

In [12]:
ddf.describe()

,Precipitation(in),LocationLat,LocationLng,ZipCode
npartitions=1,,,,
,float64,float64,float64,Float64
,...,...,...,...


In [13]:
ddf

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
npartitions=16,,,,,,,,,,,,,,
,string,string,string,string,string,float64,string,string,float64,float64,string,string,string,Int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
stime = time.time()
ddf["County"].value_counts().compute()
etime = time.time()
print("Time :", etime - stime)

Time : 27.1339054107666


In [15]:
stime = time.time()
df["County"].value_counts()
etime = time.time()
print("Time :", etime - stime)

Time : 0.3519737720489502


In [16]:
ddf["Type"].value_counts().compute()

Type
Fog              2014190
Cold              232192
Storm              61317
Precipitation     157186
Hail                2927
Rain             5000794
Snow             1158575
Name: count, dtype: int64

In [17]:
#Using Compute Group by
stime = time.time()
ddf_andrews = ddf[ddf["County"] == "Andrews"]
res_snow = ddf_andrews.groupby("Type")["EventId"].count().compute()
print(time.time() - stime)
print(res_snow)
print(type(res_snow))

19.24359941482544
Type
Fog               384
Precipitation      33
Rain             1571
Cold               59
Snow               57
Storm              21
Name: EventId, dtype: int64
<class 'pandas.core.series.Series'>


In [18]:
#Pandas Grouby
stime = time.time()
df_andrews = df[df["County"] == "Andrews"]
res_snow = df_andrews.groupby("Type")["EventId"].count()
print(time.time() - stime)
print(res_snow)

0.53141188621521
Type
Cold               59
Fog               384
Precipitation      33
Rain             1571
Snow               57
Storm              21
Name: EventId, dtype: int64


In [19]:
#Using Persist
ddf_andrews = ddf[ddf["County"] == "Andrews"].persist()
res_snow = ddf_andrews.groupby("Type")["EventId"].count().compute()
print(res_snow)
print(type(ddf_andrews))

Type
Fog               384
Precipitation      33
Rain             1571
Cold               59
Snow               57
Storm              21
Name: EventId, dtype: int64
<class 'dask_expr._collection.DataFrame'>


In [20]:
#Taking only two partitions of the data
sliced_data = ddf.partitions[3:5]
print("Sliced data no of partitions: ",sliced_data.npartitions)
print("Original data no of partitions: ",ddf.npartitions)
print("andrews data no of partitions: ",ddf.npartitions)

Sliced data no of partitions:  2
Original data no of partitions:  16
andrews data no of partitions:  16


In [21]:
#Repartition andrews data
ddf_andrews = ddf_andrews.repartition(npartitions=17)
print(ddf_andrews.npartitions)

17


In [7]:
meta = {
    'EventId': 'string',                 
    'Type': 'string',                     
    'Severity': 'string',                 
    'StartTime(UTC)': 'string',          
    'EndTime(UTC)': 'string',             
    'Precipitation(in)': 'float64',       
    'TimeZone': 'string',                
    'AirportCode': 'string',             
    'LocationLat': 'float64',            
    'LocationLng': 'float64',            
    'City': 'string',                    
    'County': 'string',                  
    'State': 'string',                   
    'ZipCode': 'Int64',
    'StartDate': "string",
    'StartTime': "string",
    'EndDate': "string",
    'EndTime': "string"
}

In [8]:
stime = time.time()
#Applying a function to all data partitions
#It took long time but was able to do it
def split_date(ddf):
    ddf[["StartDate","StartTime"]] = ddf["StartTime(UTC)"].str.split(' ',expand=True)
    ddf[["EndDate","EndTime"]] = ddf["EndTime(UTC)"].str.split(' ',expand=True)
    return ddf
ddf2 = ddf.map_partitions(split_date, meta=meta).persist()
print(time.time() - stime)

164.6466200351715


In [19]:
ddf2.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-06,23:14:00,2016-01-07,00:34:00
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-07,04:14:00,2016-01-07,04:54:00
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-07,05:54:00,2016-01-07,15:34:00
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-08,05:34:00,2016-01-08,05:54:00
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-08,13:54:00,2016-01-08,15:54:00


In [115]:
def split_date(df):
    df[["StartDate","StartTime"]] = df["StartTime(UTC)"].str.split(' ',expand=True)
    df[["EndDate","EndTime"]] = df["EndTime(UTC)"].str.split(' ',expand=True)
    return df

In [127]:
df.astype({'StartTime(UTC)': 'string',"EndTime(UTC)": "string"}).dtypes

EventId                      object
Type                         object
Severity                     object
StartTime(UTC)       string[python]
EndTime(UTC)         string[python]
Precipitation(in)           float64
TimeZone                     object
AirportCode                  object
LocationLat                 float64
LocationLng                 float64
City                         object
County                       object
State                        object
ZipCode                     float64
dtype: object

In [128]:
#Can't apply function on big data
stime = time.time()
df2 = df.apply(split_date,axis=1)
print(time.time() - stime)
df2.head()

MemoryError: Unable to allocate 197. MiB for an array with shape (3, 8627181) and data type object

In [20]:
stime = time.time()
print(ddf2.loc[:,["Severity","Type"]].compute())
print(time.time() - stime)

        Severity  Type
0          Light  Snow
1          Light  Snow
2          Light  Snow
3          Light  Snow
4          Light  Snow
...          ...   ...
535912     Light  Snow
535913  Moderate  Snow
535914    Severe  Cold
535915    Severe  Cold
535916    Severe  Cold

[8627181 rows x 2 columns]
1.4552063941955566


In [21]:
stime = time.time()
print(df.loc[:,["Severity","Type"]])
print(time.time() - stime)

         Severity  Type
0           Light  Snow
1           Light  Snow
2           Light  Snow
3           Light  Snow
4           Light  Snow
...           ...   ...
8627176     Light  Snow
8627177  Moderate  Snow
8627178    Severe  Cold
8627179    Severe  Cold
8627180    Severe  Cold

[8627181 rows x 2 columns]
0.6054530143737793


In [22]:
stime = time.time()
ddf.drop_duplicates()
print(time.time() - stime)

0.5864622592926025


In [114]:
stime = time.time()
df.drop_duplicates()
print(time.time() - stime)

100.22467613220215


In [23]:
stime = time.time()
ddf["Type"] =ddf["Type"].replace({"Snow" : "Ice"})
print(time.time() - stime)
ddf.head()

0.00699925422668457


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
1,W-2,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
2,W-3,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
3,W-4,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
4,W-5,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149


In [28]:
stime = time.time()
df["Type"] =df["Type"].replace({"Snow" : "Ice"})
print(time.time() - stime)
df.head()

0.5798912048339844


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [24]:
ddf_copy = ddf.copy()

In [25]:
stime = time.time()
eventId = ddf_copy.drop(columns="EventId")
print(time.time() - stime)

0.024402618408203125


In [26]:
type(eventId)
eventId.head()

,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
1,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
2,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
3,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
4,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149


In [27]:
eventid = ddf_copy.pop("EventId")

In [28]:
eventid.compute()

0               W-1
1               W-2
2               W-3
3               W-4
4               W-5
            ...    
535912    W-9091890
535913    W-9091891
535914    W-9091892
535915    W-9091893
535916    W-9091894
Name: EventId, Length: 8627181, dtype: object

<table>
  <tr>
    <th>Drop</th>
    <th>Pop</th>
    <th>Dropna</th>
  </tr>
  <tr>
    <td>Dropping a column without the need of it</td>
    <td>Dropping a column and return it if it is needed</td>
    <td>Dropping null values</td>
  </tr>
</table>

In [29]:
ddf_copy

,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
npartitions=16,,,,,,,,,,,,,
,string,string,string,string,float64,string,string,float64,float64,string,string,string,Int64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [30]:
ddf.astype({"Precipitation(in)": "Int64"}).dtypes

EventId              string[pyarrow]
Type                 string[pyarrow]
Severity             string[pyarrow]
StartTime(UTC)       string[pyarrow]
EndTime(UTC)         string[pyarrow]
Precipitation(in)              Int64
TimeZone             string[pyarrow]
AirportCode          string[pyarrow]
LocationLat                  float64
LocationLng                  float64
City                 string[pyarrow]
County               string[pyarrow]
State                string[pyarrow]
ZipCode                        Int64
dtype: object

In [31]:
ddf = ddf.assign(new_col= "new" )

In [32]:
ddf.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,new_col
0,W-1,Ice,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new
1,W-2,Ice,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new
2,W-3,Ice,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new
3,W-4,Ice,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new
4,W-5,Ice,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,new


In [24]:
stime = time.time()
ddf_tmp = ddf.assign(new_col2= ddf["Type"] + " " + ddf["Severity"])
print(time.time() - stime)
ddf_tmp.head()

4.489135026931763


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,new_col2
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,Snow Light
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,Snow Light
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,Snow Light
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,Snow Light
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,Snow Light


In [ ]:
ddf2.eval('new_column = Type + Severity').compute()

<h3>Pandas vs Dask Comparison</h3>
<table>
  <tr>
    <th>Assign</th>
    <th>Eval</th>
  </tr>
  <tr>
    <td>It can make implement complex functions to insert new columns</td>
    <td>It Evaluates a string to insert new column</td>
  </tr>
</table>

In [48]:
stime = time.time()
ddf_sorted = ddf.sort_values(by="Precipitation(in)", ascending=False).compute()
print(time.time() - stime)
ddf_sorted.head()

50.264055490493774


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,new_col
365804,W-3239430,Precipitation,UNK,2016-01-17 15:58:00,2016-01-19 15:58:00,1104.13,US/Eastern,KMCF,27.8493,-82.5212,Tampa,Hillsborough,FL,33621,new
216295,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,new
216263,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,new
216269,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,new
216247,W-7052976,Precipitation,UNK,2016-01-17 08:35:00,2016-02-07 14:15:00,497.97,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,new


In [49]:
stime = time.time()
df_sorted = df.sort_values(by="Precipitation(in)", ascending=False)
print(time.time() - stime)
df_sorted.head()

24.38944959640503


,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
3070926,W-3239430,Precipitation,UNK,2016-01-17 15:58:00,2016-01-19 15:58:00,1104.13,US/Eastern,KMCF,27.8493,-82.5212,Tampa,Hillsborough,FL,33621.0
6691279,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341.0
6691247,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341.0
6691253,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341.0
6691231,W-7052976,Precipitation,UNK,2016-01-17 08:35:00,2016-02-07 14:15:00,497.97,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341.0


In [37]:
ddf2["Type"].fillna(ddf2["Type"].mode).compute()

0         Snow
1         Snow
2         Snow
3         Snow
4         Snow
          ... 
535912    Snow
535913    Snow
535914    Cold
535915    Cold
535916    Cold
Name: Type, Length: 8627181, dtype: object

In [43]:
#Dask doesn't have filter
ddf2.query(" `Precipitation(in)` > 500").sort_values(by="Precipitation(in)").compute()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
216269,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-03-15,04:15:00,2016-04-06,02:15:00
216263,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-02-08,06:55:00,2016-03-02,20:15:00
216295,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-04-24,21:35:00,2016-05-23,04:55:00
365804,W-3239430,Precipitation,UNK,2016-01-17 15:58:00,2016-01-19 15:58:00,1104.13,US/Eastern,KMCF,27.8493,-82.5212,Tampa,Hillsborough,FL,33621,2016-01-17,15:58:00,2016-01-19,15:58:00


In [44]:
value = 500
ddf2.query(f"`Precipitation(in)` > {value}").sort_values(by="Precipitation(in)").compute()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
216269,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-03-15,04:15:00,2016-04-06,02:15:00
216263,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-02-08,06:55:00,2016-03-02,20:15:00
216295,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-04-24,21:35:00,2016-05-23,04:55:00
365804,W-3239430,Precipitation,UNK,2016-01-17 15:58:00,2016-01-19 15:58:00,1104.13,US/Eastern,KMCF,27.8493,-82.5212,Tampa,Hillsborough,FL,33621,2016-01-17,15:58:00,2016-01-19,15:58:00


In [46]:
ddf2.sort_values(by="Precipitation(in)").compute()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
490651,W-5639076,Fog,Severe,2019-10-20 10:04:00,2019-10-20 10:33:00,0.00,US/Central,KBVO,36.7664,-96.0129,Bartlesville,Osage,OK,74003,2019-10-20,10:04:00,2019-10-20,10:33:00
490652,W-5639077,Fog,Severe,2019-10-20 10:38:00,2019-10-20 11:06:00,0.00,US/Central,KBVO,36.7664,-96.0129,Bartlesville,Osage,OK,74003,2019-10-20,10:38:00,2019-10-20,11:06:00
490653,W-5639078,Fog,Severe,2019-10-20 11:50:00,2019-10-20 12:42:00,0.00,US/Central,KBVO,36.7664,-96.0129,Bartlesville,Osage,OK,74003,2019-10-20,11:50:00,2019-10-20,12:42:00
490654,W-5639079,Fog,Severe,2019-10-20 12:53:00,2019-10-20 15:37:00,0.00,US/Central,KBVO,36.7664,-96.0129,Bartlesville,Osage,OK,74003,2019-10-20,12:53:00,2019-10-20,15:37:00
12,W-13,Fog,Severe,2016-01-10 09:34:00,2016-01-10 10:14:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-10,09:34:00,2016-01-10,10:14:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216247,W-7052976,Precipitation,UNK,2016-01-17 08:35:00,2016-02-07 14:15:00,497.97,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-01-17,08:35:00,2016-02-07,14:15:00
216269,W-7052998,Precipitation,UNK,2016-03-15 04:15:00,2016-04-06 02:15:00,515.79,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-03-15,04:15:00,2016-04-06,02:15:00
216263,W-7052992,Precipitation,UNK,2016-02-08 06:55:00,2016-03-02 20:15:00,557.04,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-02-08,06:55:00,2016-03-02,20:15:00
216295,W-7053024,Precipitation,UNK,2016-04-24 21:35:00,2016-05-23 04:55:00,668.58,US/Central,K5H4,47.7912,-99.9317,Harvey,Wells,ND,58341,2016-04-24,21:35:00,2016-05-23,04:55:00


<h3>Pandas vs Dask Comparison</h3>
<table>
  <tr>
    <th></th>
    <th>Pandas</th>
    <th>Dask</th>
  </tr>
  <tr>
    <td>Small Dataset</td>
    <td>It will work faster as it works on the whole dataset at a time</td>
    <td>Not needed since the data is small, and as it works with partitions it will take more time</td>
  </tr>
  <tr>
    <td>Large dataset (>RAM size)</td>
    <td>Pandas will fail in speed or fail or won't be able to hold the data</td>
    <td>It will be faster and more efficient as it works in parallel using partition</td>
  </tr>
  <tr>
    <td>Methods like join, groupby, sorting..etc</td>
    <td>Fast</td>
    <td>Slow as methods that requires lots of partition reordering slows down the process</td>
  </tr>
  <tr>
    <td>Distributed data on different computers</td>
    <td>can't work with distributed data alone</td>
    <td>can work with distributed data</td>
  </tr>
</table>